# Index

In [2]:
from src.data.index_and_search import index_df, get_db_object
from loguru import logger

# run in batches to avoid memory issues - index every 100 rows in the df
def index(df_to_index, batch_size=100):
    n = len(df_to_index)
    for i in range(0, n, batch_size):
        batch_df = df_to_index.iloc[i:i+batch_size]
        print(f"--- {i} / {n} ---")
        try:
            index_df(
                df=batch_df,
                index_by_col="question_description",
                need_to_embed_col=True,
                id_col="question_description",
                collection_name="math_questions",
            )
        except Exception as e:
            logger.error(f"Error indexing batch starting at row {i}: {e}")


# Preprocess + Indexing - take first 20,000 rows from each math full dataset

In [2]:
db = get_db_object()
db.print_example("math_questions", 2)

Example from collection 'math_questions': 
([Record(id='000e2a02-ca5c-5d79-b01c-b093ecae91fb', payload={'question': "b'What is the second derivative of i**5 + 54*i**3*t + 2*i*t + 103*i + 2 wrt i?\\n'", 'answer': "b'20*i**3 + 324*i*t\\n'", 'module': 'calculus__differentiate', 'question_description': "Topic: calculus__differentiate\nQuestion: b'What is the second derivative of i**5 + 54*i**3*t + 2*i*t + 103*i + 2 wrt i?\\n'"}, vector=None, shard_key=None, order_value=None), Record(id='000e5735-9c20-5e1e-bf11-aaf35f819c0e', payload={'question': "b'Solve 27*i - 277 - 133 = -32 for i.\\n'", 'answer': "b'14\\n'", 'module': 'algebra__linear_1d', 'question_description': "Topic: algebra__linear_1d\nQuestion: b'Solve 27*i - 277 - 133 = -32 for i.\\n'"}, vector=None, shard_key=None, order_value=None)], '000fe6b6-eb2c-57f9-9a5b-b4df0a6809f3')


In [ ]:
from src.utils.folders_utils import get_repo_folder
import pandas as pd
from pathlib import Path

repo_folder = get_repo_folder()
print(f"Repository folder: {repo_folder}")


directory = Path(repo_folder / "src/data/DB_questions/Math/math_dataset_csvs")

dfs = []

# Loop through all CSV files
for file_path in directory.glob("*.csv"):
    if "algebra__linear_1d_full.csv" in file_path.name or "mini" in file_path.name \
        or "algebra__polynomial_roots_full" in file_path.name:
        continue
    print(f"Processing {file_path.name}...")

    df = pd.read_csv(file_path)[23000:25000]

    df["question_description"] = df.apply(lambda row: f"Topic: {row['module']}\nQuestion: {row['question']}", axis=1)

    index(df)

Repository folder: /Users/kereng/Projects/PrivateTeacherAgent
Processing numbers__gcd_full.csv...
--- 0 / 2000 ---


Inserting data into math_questions: 100%|██████████| 100/100 [00:00<00:00, 44407.67it/s]


Collection 'math_questions' size: count=33405
--- 100 / 2000 ---


Inserting data into math_questions: 100%|██████████| 100/100 [00:00<00:00, 34781.52it/s]


Collection 'math_questions' size: count=33505
--- 200 / 2000 ---


...

Inserting data into math_questions: 100%|██████████| 100/100 [00:00<00:00, 11507.64it/s]


Collection 'math_questions' size: count=41405
--- 100 / 2000 ---


Inserting data into math_questions: 100%|██████████| 100/100 [00:00<00:00, 35738.79it/s]


Collection 'math_questions' size: count=41505
--- 200 / 2000 ---


Inserting data into math_questions: 100%|██████████| 100/100 [00:00<00:00, 48953.13it/s]


In [ ]:
dfs[0]
